In [1]:
import os
# os.chdir('../')
%pwd

'c:\\Users\\akish.pothuri\\python\\LangChain\\experiment'

In [1]:
import pyodbc
cnxn = pyodbc.connect(r'Driver=SQL Server;Server=GGKU4DELL1254;Database=projectcodefirst;Trusted_Connection=yes;')

In [2]:
cnxn.cursor()

In [3]:
cursor = cnxn.cursor()
cursor.execute("SELECT * FROM Employees")

In [4]:
row = cursor.fetchall()

In [5]:
row

[(1, 'Madhu', 1), (2, 'Anil', 2), (3, 'Amir', 3), (7, 'Akbar', 3)]

In [ ]:
# pip install google-generativeai
#pip install python-dotenv

In [7]:
import os
from langchain.llms import GooglePalm
from dotenv import load_dotenv 
load_dotenv()


True

In [10]:
palm_key = os.getenv('api_key')

In [11]:
llm = GooglePalm(google_api_key=palm_key, temperature=0.3)

In [12]:
code = llm("Code to linear regression in machine learning algorithms")

c:\Users\akish.pothuri\python\LangChain\SQl_Script_chatbot\chatbot\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [13]:
print(code)

```python
import numpy as np
import matplotlib.pyplot as plt

# Generate data
x = np.random.rand(100)
y = 2 * x + np.random.randn(100)

# Plot data
plt.scatter(x, y)
plt.show()

# Fit linear regression model
w, b = np.linalg.lstsq(x.reshape(-1, 1), y.reshape(-1, 1))[0]

# Plot fitted line
y_pred = w * x + b
plt.plot(x, y_pred)
plt.show()
```


In [14]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.prompts.chat import ChatPromptTemplate
from sqlalchemy import create_engine

In [15]:
# check the local ODBC driver and make sure it match with the traget database instance
import pyodbc
for driver in pyodbc.drivers():
    print(driver)

SQL Server
SQL Server Native Client 11.0
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server


In [16]:
#windows authentication connection string.
db = SQLDatabase.from_uri(
    database_uri="mssql+pyodbc://GGKU4DELL1254/shopping?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)

In [17]:
sql_toolkit=SQLDatabaseToolkit(db=db,llm=llm)
sql_toolkit.get_tools()

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000018817DBED00>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000018817DBED00>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000018817DBED00>),
 QuerySQLCheckerTool(description='Use this tool to 

In [18]:
print(db.get_table_names())

['Customer', 'OrderItem', 'Orders', 'Product', 'Supplier']


c:\Users\akish.pothuri\python\LangChain\SQl_Script_chatbot\chatbot\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `get_table_names` was deprecated in LangChain 0.0.1 and will be removed in 0.2.0. Use get_usable_table_name instead.
  warn_deprecated(


In [19]:
from langchain_experimental.sql import SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("what are the total number of customers")

c:\Users\akish.pothuri\python\LangChain\SQl_Script_chatbot\chatbot\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
what are the total number of customers
SQLQuery:SELECT COUNT(*) FROM Customer
SQLResult: [(91,)]
Answer:91
> Finished chain.


In [20]:
from langchain_experimental.sql import SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns2 = db_chain("what are the names of the customers from germany courntry?")



> Entering new SQLDatabaseChain chain...
what are the names of the customers from germany courntry?
SQLQuery:SELECT FirstName, LastName FROM Customer WHERE Country = 'Germany'
SQLResult: [('Maria', 'Anders'), ('Hanna', 'Moos'), ('Sven', 'Ottlieb'), ('Peter', 'Franken'), ('Philip', 'Cramer'), ('Renate', 'Messner'), ('Alexander', 'Feuer'), ('Henriette', 'Pfalzheim'), ('Horst', 'Kloss'), ('Karin', 'Josephs'), ('Rita', 'Müller')]
Answer:Maria Anders, Hanna Moos, Sven Ottlieb, Peter Franken, Philip Cramer, Renate Messner, Alexander Feuer, Henriette Pfalzheim, Horst Kloss, Karin Josephs, Rita Müller
> Finished chain.


In [23]:
qns3 = db_chain("what is the name of Supplier providing more products?")




> Entering new SQLDatabaseChain chain...
what is the name of Supplier providing more products?
SQLQuery:SELECT CompanyName FROM Supplier s JOIN Product p ON s.Id = p.SupplierId GROUP BY CompanyName ORDER BY COUNT(*) DESC
SQLResult: [('Pavlova, Ltd.',), ('Plutzer Lebensmittelgroßmärkte AG',), ('Specialty Biscuits, Ltd.',), ('New Orleans Cajun Delights',), ('Norske Meierier',), ('Formaggi Fortini s.r.l.',), ("Mayumi's",), ('Bigfoot Breweries',), ('Exotic Liquids',), ("G'day, Mate",), ("Grandma Kelly's Homestead",), ('Heli Süßwaren GmbH & Co. KG',), ('Karkki Oy',), ('Leka Trading',), ('Svensk Sjöföda AB',), ('Tokyo Traders',), ('Zaanse Snoepfabriek',), ('PB Knäckebröd AB',), ('Aux joyeux ecclésiastiques',), ('Lyngbysild',), ('Ma Maison',), ("Forêts d'érables",), ("Cooperativa de Quesos 'Las Cabras'",), ('New England Seafood Cannery',), ('Gai pâturage',), ('Pasta Buttini s.r.l.',), ('Nord-Ost-Fisch Handelsgesellschaft mbH',), ('Escargots Nouveaux',), ('Refrescos Americanas LTDA',)]
Answe

In [24]:
qns3['result']

'Pavlova, Ltd.'

In [26]:
qns4 = db_chain("what is the name of customer ordered maximum quantity according to the data in the database?")



> Entering new SQLDatabaseChain chain...
what is the name of customer ordered maximum quantity according to the data in the database?
SQLQuery:SELECT TOP 1 c.FirstName + ' ' + c.LastName AS CustomerName 
FROM Customer AS c 
INNER JOIN Orders AS o ON c.Id = o.CustomerId 
INNER JOIN OrderItem AS i ON o.Id = i.OrderId 
GROUP BY c.Id, c.FirstName, c.LastName 
ORDER BY SUM(i.Quantity) DESC
SQLResult: [('Jose Pavarotti',)]
Answer:Jose Pavarotti
> Finished chain.


In [43]:
qns5 = db_chain("Provide me names of customers according to their respective countries")
# print(qns5)



> Entering new SQLDatabaseChain chain...
Provide me names of customers according to their respective countries
SQLQuery:SELECT c.FirstName, c.LastName, c.Country FROM Customer c
SQLResult: [('Maria', 'Anders', 'Germany'), ('Ana', 'Trujillo', 'Mexico'), ('Antonio', 'Moreno', 'Mexico'), ('Thomas', 'Hardy', 'UK'), ('Christina', 'Berglund', 'Sweden'), ('Hanna', 'Moos', 'Germany'), ('Frédérique', 'Citeaux', 'France'), ('Martín', 'Sommer', 'Spain'), ('Laurence', 'Lebihan', 'France'), ('Elizabeth', 'Lincoln', 'Canada'), ('Victoria', 'Ashworth', 'UK'), ('Patricio', 'Simpson', 'Argentina'), ('Francisco', 'Chang', 'Mexico'), ('Yang', 'Wang', 'Switzerland'), ('Pedro', 'Afonso', 'Brazil'), ('Elizabeth', 'Brown', 'UK'), ('Sven', 'Ottlieb', 'Germany'), ('Janine', 'Labrune', 'France'), ('Ann', 'Devon', 'UK'), ('Roland', 'Mendel', 'Austria'), ('Aria', 'Cruz', 'Brazil'), ('Diego', 'Roel', 'Spain'), ('Martine', 'Rancé', 'France'), ('Maria', 'Larsson', 'Sweden'), ('Peter', 'Franken', 'Germany'), ('Cari

'**Original:**\n\nThe quick brown fox jumps over the lazy dog.\n\n**Paraphrased:**\n\nThe nimble fox swiftly leaps over the lethargic canine.'

In [49]:
answer = llm("play role of paraphraser, arrange and group this data {} into a understandable format".format(qns5['result']))
print(answer)

| Name | Country |
|---|---|
| Maria Anders | Germany |
| Ana Trujillo | Mexico |
| Antonio Moreno | Mexico |
| Thomas Hardy | UK |
| Christina Berglund | Sweden |
| Hanna Moos | Germany |
| Frédérique Citeaux | France |
| Martín Sommer | Spain |
| Laurence Lebihan | France |
| Elizabeth Lincoln | Canada |
| Victoria Ashworth | UK |
| Patricio Simpson | Argentina |
| Francisco Chang | Mexico |
| Yang Wang | Switzerland |
| Pedro Afonso | Brazil |
| Elizabeth Brown | UK |
| Sven Ottlieb | Germany |
| Janine Labrune | France |
| Ann Devon | UK |
| Roland Mendel | Austria |
| Aria Cruz | Brazil |
| Diego Roel | Spain |
| Martine Rancé | France |
| Maria Larsson | Sweden |
| Peter Franken | Germany |
| Carine Schmitt | France |
| Paolo Accorti | Italy |
| Lino Rodriguez | Portugal |
| Eduardo Saavedra | Spain |
| José Pedro Freyre | Spain |
| André Fonseca | Brazil |
| Howard Snyder | USA |
| Manuel Pereira | Venezuela |
| Mario Pontes | Brazil |
| Carlos Hernández | Venezuela |
| Yoshi La

In [51]:
type(qns5['result'])

str

In [41]:
print(qns5['result'])

Maria Anders (Germany), Ana Trujillo (Mexico), Antonio Moreno (Mexico), Thomas Hardy (UK), Christina Berglund (Sweden), Hanna Moos (Germany), Frédérique Citeaux (France), Martín Sommer (Spain), Laurence Lebihan (France), Elizabeth Lincoln (Canada), Victoria Ashworth (UK), Patricio Simpson (Argentina), Francisco Chang (Mexico), Yang Wang (Switzerland), Pedro Afonso (Brazil), Elizabeth Brown (UK), Sven Ottlieb (Germany), Janine Labrune (France), Ann Devon (UK), Roland Mendel (Austria), Aria Cruz (Brazil), Diego Roel (Spain), Martine Rancé (France), Maria Larsson (Sweden), Peter Franken (Germany), Carine Schmitt (France), Paolo Accorti (Italy), Lino Rodriguez (Portugal), Eduardo Saavedra (Spain), José Pedro Freyre (Spain), André Fonseca (Brazil), Howard Snyder (USA), Manuel Pereira (Venezuela), Mario Pontes (Brazil), Carlos Hernández (Venezuela), Yoshi Latimer (USA), Patricia McKenna (Ireland), Helen Bennett (UK), Philip Cramer (Germany), Daniel Tonini (France), Annette Roulet (France), Y

In [42]:
qns6 = db_chain.run("SELECT Country, FirstName + ' ' + LastName FROM Customer GROUP BY Country, FirstName, LastName;")



> Entering new SQLDatabaseChain chain...
SELECT Country, FirstName + ' ' + LastName FROM Customer GROUP BY Country, FirstName, LastName;
SQLQuery:SELECT Country, FirstName + ' ' + LastName FROM Customer GROUP BY Country, FirstName, LastName
SQLResult: [('Argentina', 'Patricio Simpson'), ('Argentina', 'Sergio Gutiérrez'), ('Argentina', 'Yvonne Moncada'), ('Austria', 'Georg Pipps'), ('Austria', 'Roland Mendel'), ('Belgium', 'Catherine Dewey'), ('Belgium', 'Pascale Cartrain'), ('Brazil', 'Anabela Domingues'), ('Brazil', 'André Fonseca'), ('Brazil', 'Aria Cruz'), ('Brazil', 'Bernardo Batista'), ('Brazil', 'Janete Limeira'), ('Brazil', 'Lúcia Carvalho'), ('Brazil', 'Mario Pontes'), ('Brazil', 'Paula Parente'), ('Brazil', 'Pedro Afonso'), ('Canada', 'Elizabeth Lincoln'), ('Canada', 'Jean Fresnière'), ('Canada', 'Yoshi Tannamuri'), ('Denmark', 'Jytte Petersen'), ('Denmark', 'Palle Ibsen'), ('Finland', 'Matti Karttunen'), ('Finland', 'Pirkko Koskitalo'), ('France', 'Annette Roulet'), ('Franc

In [69]:
qns6

'Argentina: Patricio Simpson, Sergio Gutiérrez, Yvonne Moncada\nAustria: Georg Pipps, Roland Mendel\nBelgium: Catherine Dewey, Pascale Cartrain\nBrazil: Anabela Domingues, André Fonseca, Aria Cruz, Bernardo Batista, Janete Limeira, Lúcia Carvalho, Mario Pontes, Paula Parente, Pedro Afonso\nCanada: Elizabeth Lincoln, Jean Fresnière, Yoshi Tannamuri\nDenmark: Jytte Petersen, Palle Ibsen\nFinland: Matti Karttunen, Pirkko Koskitalo\nFrance: Annette Roulet, Carine Schmitt, Daniel Tonini, Dominique Perrier, Frédérique Citeaux, Janine Labrune, Laurence Lebihan, Marie Bertrand, Martine Rancé, Mary Saveley, Paul Henriot\nGermany: Alexander Feuer, Hanna Moos, Henriette Pfalzheim, Horst Kloss, Karin Josephs, Maria Anders, Peter Franken, Philip Cramer, Renate Messner, Rita Müller, Sven Ottlieb\nIreland: Patricia McKenna\nItaly: Giovanni Rovelli, Maurizio Moroni, Paolo Accorti\nMexico: Ana Trujillo, Antonio Moreno, Francisco Chang, Guillermo Fernández, Miguel Angel Paolino\nNorway: Jonas Bergulfsen

In [29]:
qns7 = db_chain("Give me the date on we got maximum amount?")



> Entering new SQLDatabaseChain chain...
Give me the date on we got maximum amount?
SQLQuery:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


SELECT OrderDate FROM Orders ORDER BY TotalAmount DESC OFFSET 0 ROWS FETCH NEXT 1 ROWS ONLY
SQLResult: [(datetime.datetime(2014, 2, 2, 0, 0),)]
Answer:2014-02-02
> Finished chain.


In [30]:
qns8 = db_chain("Give me the date on we got minimum amount?")



> Entering new SQLDatabaseChain chain...
Give me the date on we got minimum amount?
SQLQuery:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


SELECT MIN(OrderDate) FROM Orders
SQLResult: [(datetime.datetime(2012, 7, 4, 0, 0),)]
Answer:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


2012-07-04
> Finished chain.


## Few Shot

In [125]:
few_shots = [
    {
        'Question' : "what is the name of Supplier providing more products",
        'SQLQuery' : "SELECT CompanyName FROM Supplier s JOIN Product p ON s.Id = p.SupplierId GROUP BY CompanyName ORDER BY COUNT(*) DESC",
        'SQLResult' : """[('Pavlova, Ltd.',), ('Plutzer Lebensmittelgroßmärkte AG',), ('Specialty Biscuits, Ltd.',), ('New Orleans Cajun Delights',), ('Norske Meierier',), ('Formaggi Fortini s.r.l.',), ("Mayumi's",), ('Bigfoot Breweries',), ('Exotic Liquids',), ("G'day, Mate",), ("Grandma Kelly's Homestead",), ('Heli Süßwaren GmbH & Co. KG',), ('Karkki Oy',), ('Leka Trading',), ('Svensk Sjöföda AB',), ('Tokyo Traders',), ('Zaanse Snoepfabriek',), ('PB Knäckebröd AB',), ('Aux joyeux ecclésiastiques',), ('Lyngbysild',), ('Ma Maison',), ("Forêts d'érables",), ("Cooperativa de Quesos 'Las Cabras'",), ('New England Seafood Cannery',), ('Gai pâturage',), ('Pasta Buttini s.r.l.',), ('Nord-Ost-Fisch Handelsgesellschaft mbH',), ('Escargots Nouveaux',), ('Refrescos Americanas LTDA',)]""",
        'Answer' : "Pavlova, Ltd"
    },
    {
        'Question' : "Give me the date on we got minimum amount",
        'SQLQuery' : "SELECT MIN(OrderDate) FROM Orders",
        'SQLResult' : "[(datetime.datetime(2012, 7, 4, 0, 0),)]",
        'Answer' : "2012-07-04"
    },
    {
        'Question' : "Provide me names of customers according to their respective countries",
        'SQLQuery' : "SELECT Country, FirstName + ' ' + LastName FROM Customer GROUP BY Country, FirstName, LastName;",
        'SQLResult' : "[('Argentina', 'Patricio Simpson'), ('Argentina', 'Sergio Gutiérrez'), ('Argentina', 'Yvonne Moncada'), ('Austria', 'Georg Pipps'), ('Austria', 'Roland Mendel'), ('Belgium', 'Catherine Dewey'), ('Belgium', 'Pascale Cartrain'), ('Brazil', 'Anabela Domingues'), ('Brazil', 'André Fonseca'), ('Brazil', 'Aria Cruz'), ('Brazil', 'Bernardo Batista'), ('Brazil', 'Janete Limeira'), ('Brazil', 'Lúcia Carvalho'), ('Brazil', 'Mario Pontes'), ('Brazil', 'Paula Parente'), ('Brazil', 'Pedro Afonso'), ('Canada', 'Elizabeth Lincoln'), ('Canada', 'Jean Fresnière'), ('Canada', 'Yoshi Tannamuri'), ('Denmark', 'Jytte Petersen'), ('Denmark', 'Palle Ibsen'), ('Finland', 'Matti Karttunen'), ('Finland', 'Pirkko Koskitalo'), ('France', 'Annette Roulet'), ('France', 'Carine Schmitt'), ('France', 'Daniel Tonini'), ('France', 'Dominique Perrier'), ('France', 'Frédérique Citeaux'), ('France', 'Janine Labrune'), ('France', 'Laurence Lebihan'), ('France', 'Marie Bertrand'), ('France', 'Martine Rancé'), ('France', 'Mary Saveley'), ('France', 'Paul Henriot'), ('Germany', 'Alexander Feuer'), ('Germany', 'Hanna Moos'), ('Germany', 'Henriette Pfalzheim'), ('Germany', 'Horst Kloss'), ('Germany', 'Karin Josephs'), ('Germany', 'Maria Anders'), ('Germany', 'Peter Franken'), ('Germany', 'Philip Cramer'), ('Germany', 'Renate Messner'), ('Germany', 'Rita Müller'), ('Germany', 'Sven Ottlieb'), ('Ireland', 'Patricia McKenna'), ('Italy', 'Giovanni Rovelli'), ('Italy', 'Maurizio Moroni'), ('Italy', 'Paolo Accorti'), ('Mexico', 'Ana Trujillo'), ('Mexico', 'Antonio Moreno'), ('Mexico', 'Francisco Chang'), ('Mexico', 'Guillermo Fernández'), ('Mexico', 'Miguel Angel Paolino'), ('Norway', 'Jonas Bergulfsen'), ('Poland', 'Zbyszek Piestrzeniewicz'), ('Portugal', 'Isabel de Castro'), ('Portugal', 'Lino Rodriguez'), ('Spain', 'Alejandra Camino'), ('Spain', 'Diego Roel'), ('Spain', 'Eduardo Saavedra'), ('Spain', 'José Pedro Freyre'), ('Spain', 'Martín Sommer'), ('Sweden', 'Christina Berglund'), ('Sweden', 'Maria Larsson'), ('Switzerland', 'Michael Holz'), ('Switzerland', 'Yang Wang'), ('UK', 'Ann Devon'), ('UK', 'Elizabeth Brown'), ('UK', 'Hari Kumar'), ('UK', 'Helen Bennett'), ('UK', 'Simon Crowther'), ('UK', 'Thomas Hardy'), ('UK', 'Victoria Ashworth'), ('USA', 'Art Braunschweiger'), ('USA', 'Fran Wilson'), ('USA', 'Helvetius Nagy'), ('USA', 'Howard Snyder'), ('USA', 'Jaime Yorres'), ('USA', 'John Steel'), ('USA', 'Jose Pavarotti'), ('USA', 'Karl Jablonski'), ('USA', 'Liu Wong'), ('USA', 'Liz Nixon'), ('USA', 'Paula Wilson'), ('USA', 'Rene Phillips'), ('USA', 'Yoshi Latimer'), ('Venezuela', 'Carlos González'), ('Venezuela', 'Carlos Hernández'), ('Venezuela', 'Felipe Izquierdo'), ('Venezuela', 'Manuel Pereira')]",
        'Answer' : """Argentina: Patricio Simpson, Sergio Gutiérrez, Yvonne Moncada
                      Austria: Georg Pipps, Roland Mendel
                      Belgium: Catherine Dewey, Pascale Cartrain
                      Brazil: Anabela Domingues, André Fonseca, Aria Cruz, Bernardo Batista, Janete Limeira, Lúcia Carvalho, Mario Pontes, Paula Parente, Pedro Afonso
                      Canada: Elizabeth Lincoln, Jean Fresnière, Yoshi Tannamuri
                      Denmark: Jytte Petersen, Palle Ibsen
                      Finland: Matti Karttunen, Pirkko Koskitalo
                      France: Annette Roulet, Carine Schmitt, Daniel Tonini, Dominique Perrier, Frédérique Citeaux, Janine Labrune, Laurence Lebihan, Marie Bertrand, Martine Rancé, Mary Saveley, Paul Henriot
                      Germany: Alexander Feuer, Hanna Moos, Henriette Pfalzheim, Horst Kloss, Karin Josephs, Maria Anders, Peter Franken, Philip Cramer, Renate Messner, Rita Müller, Sven Ottlieb
                      Ireland: Patricia McKenna
                      Italy: Giovanni Rovelli, Maurizio Moroni, Paolo Accorti
                      Mexico: Ana Trujillo, Antonio Moreno, Francisco Chang, Guillermo Fernández, Miguel Angel Paolino
                      Norway: Jonas Bergulfsen
                      Poland: Zbyszek Piestrzeniewicz
                      Portugal: Isabel de Castro, Lino Rodriguez
                      Spain: Alejandra Camino, Diego Roel, Eduardo Saavedra, José Pedro Freyre, Martín Sommer
                      Sweden: Christina Berglund, Maria Larsson
                      Switzerland: Michael Holz, Yang Wang
                      UK: Ann Devon, Elizabeth Brown, Hari Kumar, Helen Bennett, Simon Crowther, Thomas Hardy, Victoria Ashworth
                      USA: Art Braunschweiger, Fran Wilson, Helvetius Nagy, Howard Snyder, Jaime Yorres, John Steel, Jose Pavarotti, Karl Jablonski, Liu Wong, Liz Nixon, Paula Wilson, Rene Phillips, Yoshi Latimer
                      Venezuela: Carlos González, Carlos Hernández, Felipe Izquierdo, Manuel Pereira"""
    },
    {
        'Question' : "Name of the suppliers according to the Country wise",
        'SQLQuery' : "SELECT Country,CompanyName FROM Supplier GROUP BY Country,CompanyName ORDER BY Country;",
        'SQLResult' : """ [('Australia', "G'day, Mate"), ('Australia', 'Pavlova, Ltd.'), ('Brazil', 'Refrescos Americanas LTDA'), ('Canada', "Forêts d'érables"), ('Canada', 'Ma Maison'), ('Denmark', 'Lyngbysild'), ('Finland', 'Karkki Oy'), ('France', 'Aux joyeux ecclésiastiques'), ('France', 'Escargots Nouveaux'), ('France', 'Gai pâturage'), ('Germany', 'Heli Süßwaren GmbH & Co. KG'), ('Germany', 'Nord-Ost-Fisch Handelsgesellschaft mbH'), ('Germany', 'Plutzer Lebensmittelgroßmärkte AG'), ('Italy', 'Formaggi Fortini s.r.l.'), ('Italy', 'Pasta Buttini s.r.l.'), ('Japan', "Mayumi's"), ('Japan', 'Tokyo Traders'), ('Netherlands', 'Zaanse Snoepfabriek'), ('Norway', 'Norske Meierier'), ('Singapore', 'Leka Trading'), ('Spain', "Cooperativa de Quesos 'Las Cabras'"), ('Sweden', 'PB Knäckebröd AB'), ('Sweden', 'Svensk Sjöföda AB'), ('UK', 'Exotic Liquids'), ('UK', 'Specialty Biscuits, Ltd.'), ('USA', 'Bigfoot Breweries'), ('USA', "Grandma Kelly's Homestead"), ('USA', 'New England Seafood Cannery'), ('USA', 'New Orleans Cajun Delights')]""",
        'Answer' :  """Australia, G'day, Mate | Australia, Pavlova, Ltd. | Brazil, Refrescos Americanas LTDA | Canada, "Forêts d'érables" | Canada, Ma Maison | Denmark, Lyngbysild | Finland, Karkki Oy | France, Aux joyeux ecclésiastiques | France, Escargots Nouveaux | France, Gai pâturage | Germany, Heli Süßwaren GmbH & Co. KG | Germany, Nord-Ost-Fisch Handelsgesellschaft mbH | Germany, Plutzer Lebensmittelgroßmärkte AG | Italy, Formaggi Fortini s.r.l. | Italy, Pasta Buttini s.r.l. | Japan, "Mayumi's" | Japan, Tokyo Traders | Netherlands, Zaanse Snoepfabriek | Norway, Norske Meierier | Singapore, Leka Trading | Spain, "Cooperativa de Quesos 'Las Cabras'" | Sweden, PB Knäckebröd AB | Sweden, Svensk Sjöföda AB | UK, Exotic Liquids | UK, Specialty Biscuits, Ltd. | USA, Bigfoot Breweries | USA, "Grandma Kelly's Homestead" | USA, New England Seafood Cannery | USA, New Orleans Cajun Delights"""
    },
    {
        'Question' : "what is the name of customer ordered maximum quantity according to the data in the database",
        'SQLQuery' : "SELECT TOP 1 c.FirstName + ' ' + c.LastName AS CustomerName  FROM Customer AS c INNER JOIN Orders AS o ON c.Id = o.CustomerId INNER JOIN OrderItem AS i ON o.Id = i.OrderId GROUP BY c.Id, c.FirstName, c.LastName ORDER BY SUM(i.Quantity) DESC",
        'SQLResult' : "[('Jose Pavarotti',)]",
        'Answer' : "Jose Pavarotti"
    },
    {
        'Question' : "Total number of suppliers",
        'SQLQuery' : "SELECT COUNT(*) FROM Supplier",
        'SQLResult' : "[(29,)]",
        'Answer' : "29"
    },
    {
        'Question' : "How many products discontinued till now?",
        'SQLQuery' : "SELECT COUNT(*) FROM Product WHERE IsDiscontinued = 1",
        'SQLResult' : "[(8,)]",
        'Answer' : "8"
    }

]

In [57]:
qns9 = db_chain("Total number of suppliers")



> Entering new SQLDatabaseChain chain...
Total number of suppliers
SQLQuery:SELECT COUNT(*) FROM Supplier
SQLResult: [(29,)]
Answer:29
> Finished chain.


In [60]:
qns10 = db_chain("Name of the suppliers according to the Country wise")



> Entering new SQLDatabaseChain chain...
Name of the suppliers according to the Country wise
SQLQuery:SELECT CompanyName, Country FROM Supplier GROUP BY Country

ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Column 'Supplier.CompanyName' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause. (8120) (SQLExecDirectW)")
[SQL: SELECT CompanyName, Country FROM Supplier GROUP BY Country]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [79]:
qns10sql = db_chain.run("SELECT Country,CompanyName FROM Supplier GROUP BY Country,CompanyName ORDER BY Country;")



> Entering new SQLDatabaseChain chain...
 SELECT Country,CompanyName FROM Supplier GROUP BY Country,CompanyName ORDER BY Country;
SQLQuery:SELECT Country,CompanyName FROM Supplier GROUP BY Country,CompanyName ORDER BY Country;
SQLResult: [('Australia', "G'day, Mate"), ('Australia', 'Pavlova, Ltd.'), ('Brazil', 'Refrescos Americanas LTDA'), ('Canada', "Forêts d'érables"), ('Canada', 'Ma Maison'), ('Denmark', 'Lyngbysild'), ('Finland', 'Karkki Oy'), ('France', 'Aux joyeux ecclésiastiques'), ('France', 'Escargots Nouveaux'), ('France', 'Gai pâturage'), ('Germany', 'Heli Süßwaren GmbH & Co. KG'), ('Germany', 'Nord-Ost-Fisch Handelsgesellschaft mbH'), ('Germany', 'Plutzer Lebensmittelgroßmärkte AG'), ('Italy', 'Formaggi Fortini s.r.l.'), ('Italy', 'Pasta Buttini s.r.l.'), ('Japan', "Mayumi's"), ('Japan', 'Tokyo Traders'), ('Netherlands', 'Zaanse Snoepfabriek'), ('Norway', 'Norske Meierier'), ('Singapore', 'Leka Trading'), ('Spain', "Cooperativa de Quesos 'Las Cabras'"), ('Sweden', 'PB Knä

In [126]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

to_vectorize = [" ".join(example.values()) for example in few_shots]

In [127]:
to_vectorize

['what is the name of Supplier providing more products SELECT CompanyName FROM Supplier s JOIN Product p ON s.Id = p.SupplierId GROUP BY CompanyName ORDER BY COUNT(*) DESC [(\'Pavlova, Ltd.\',), (\'Plutzer Lebensmittelgroßmärkte AG\',), (\'Specialty Biscuits, Ltd.\',), (\'New Orleans Cajun Delights\',), (\'Norske Meierier\',), (\'Formaggi Fortini s.r.l.\',), ("Mayumi\'s",), (\'Bigfoot Breweries\',), (\'Exotic Liquids\',), ("G\'day, Mate",), ("Grandma Kelly\'s Homestead",), (\'Heli Süßwaren GmbH & Co. KG\',), (\'Karkki Oy\',), (\'Leka Trading\',), (\'Svensk Sjöföda AB\',), (\'Tokyo Traders\',), (\'Zaanse Snoepfabriek\',), (\'PB Knäckebröd AB\',), (\'Aux joyeux ecclésiastiques\',), (\'Lyngbysild\',), (\'Ma Maison\',), ("Forêts d\'érables",), ("Cooperativa de Quesos \'Las Cabras\'",), (\'New England Seafood Cannery\',), (\'Gai pâturage\',), (\'Pasta Buttini s.r.l.\',), (\'Nord-Ost-Fisch Handelsgesellschaft mbH\',), (\'Escargots Nouveaux\',), (\'Refrescos Americanas LTDA\',)] Pavlova, Ltd'

In [128]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [137]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "Get me Name of the suppliers according to the Country wise"})

[{'Answer': 'Australia, G\'day, Mate | Australia, Pavlova, Ltd. | Brazil, Refrescos Americanas LTDA | Canada, "Forêts d\'érables" | Canada, Ma Maison | Denmark, Lyngbysild | Finland, Karkki Oy | France, Aux joyeux ecclésiastiques | France, Escargots Nouveaux | France, Gai pâturage | Germany, Heli Süßwaren GmbH & Co. KG | Germany, Nord-Ost-Fisch Handelsgesellschaft mbH | Germany, Plutzer Lebensmittelgroßmärkte AG | Italy, Formaggi Fortini s.r.l. | Italy, Pasta Buttini s.r.l. | Japan, "Mayumi\'s" | Japan, Tokyo Traders | Netherlands, Zaanse Snoepfabriek | Norway, Norske Meierier | Singapore, Leka Trading | Spain, "Cooperativa de Quesos \'Las Cabras\'" | Sweden, PB Knäckebröd AB | Sweden, Svensk Sjöföda AB | UK, Exotic Liquids | UK, Specialty Biscuits, Ltd. | USA, Bigfoot Breweries | USA, "Grandma Kelly\'s Homestead" | USA, New England Seafood Cannery | USA, New Orleans Cajun Delights',
  'Question': 'Name of the suppliers according to the Country wise',
  'SQLQuery': 'SELECT Country,Comp

In [130]:
### my sql based instruction prompt
mysql_prompt = """You are a Microsoft SQL Server Management Studio 2019 expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table. use TOP instead of LIMIT.
Pay attention to use GetDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [131]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


## Setting up PromptTemplete using input variables

In [132]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [133]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [134]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [114]:
new_chain("How many Customers are there from Germany?")



> Entering new SQLDatabaseChain chain...
How many Customers are there from Germany?
SQLQuery:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


SELECT COUNT(*) FROM Customer WHERE Country = 'Germany'
SQLResult: [(11,)]
Answer:11
> Finished chain.


{'query': 'How many Customers are there from Germany?', 'result': '11'}

In [121]:
new_chain("What is the date today?")



> Entering new SQLDatabaseChain chain...
What is the date today?
SQLQuery:SELECT GETDATE()
SQLResult: [(datetime.datetime(2024, 2, 16, 15, 20, 6, 947000),)]
Answer:2024-02-16 15:20:06.947000
> Finished chain.


{'query': 'What is the date today?', 'result': '2024-02-16 15:20:06.947000'}

In [123]:
new_chain("How many products discontinued till now?")



> Entering new SQLDatabaseChain chain...
How many products discontinued till now?
SQLQuery:SELECT COUNT(*) FROM Product WHERE IsDiscontinued = 1
SQLResult: [(8,)]
Answer:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


8
> Finished chain.


{'query': 'How many products discontinued till now?', 'result': '8'}

In [140]:
new_chain("what are the names of the suppliers according to the Country wise")



> Entering new SQLDatabaseChain chain...
what are the names of the suppliers according to the Country wise
SQLQuery:SELECT Country, CompanyName FROM Supplier GROUP BY Country, CompanyName;
SQLResult: [('Australia', "G'day, Mate"), ('Australia', 'Pavlova, Ltd.'), ('Brazil', 'Refrescos Americanas LTDA'), ('Canada', "Forêts d'érables"), ('Canada', 'Ma Maison'), ('Denmark', 'Lyngbysild'), ('Finland', 'Karkki Oy'), ('France', 'Aux joyeux ecclésiastiques'), ('France', 'Escargots Nouveaux'), ('France', 'Gai pâturage'), ('Germany', 'Heli Süßwaren GmbH & Co. KG'), ('Germany', 'Nord-Ost-Fisch Handelsgesellschaft mbH'), ('Germany', 'Plutzer Lebensmittelgroßmärkte AG'), ('Italy', 'Formaggi Fortini s.r.l.'), ('Italy', 'Pasta Buttini s.r.l.'), ('Japan', "Mayumi's"), ('Japan', 'Tokyo Traders'), ('Netherlands', 'Zaanse Snoepfabriek'), ('Norway', 'Norske Meierier'), ('Singapore', 'Leka Trading'), ('Spain', "Cooperativa de Quesos 'Las Cabras'"), ('Sweden', 'PB Knäckebröd AB'), ('Sweden', 'Svensk Sjöfö

{'query': 'what are the names of the suppliers according to the Country wise',
 'result': 'Australia, G\'day, Mate | Australia, Pavlova, Ltd. | Brazil, Refrescos Americanas LTDA | Canada, "Forêts d\'érables" | Canada, Ma Maison | Denmark, Lyngbysild | Finland, Karkki Oy | France, Aux joyeux ecclésiastiques | France, Escargots Nouveaux | France, Gai pâturage | Germany, Heli Süßwaren GmbH & Co. KG | Germany, Nord-Ost-Fisch Handelsgesellschaft mbH | Germany, Plutzer Lebensmittelgroßmärkte AG | Italy, Formaggi Fortini s.r.l. | Italy, Pasta Buttini s.r.l. | Japan, "Mayumi\'s" | Japan, Tokyo Traders | Netherlands, Zaanse Snoepfabriek | Norway, Norske Meierier | Singapore, Leka Trading | Spain, "Cooperativa de Quesos \'Las Cabras\'" | Sweden, PB Knäckebröd AB | Sweden, Svensk Sjöföda AB | UK, Exotic Liquids | UK, Specialty Biscuits, Ltd. | USA, Bigfoot Breweries | USA, "Grandma Kelly\'s Homestead" | USA, New England Seafood Cannery | USA, New Orleans Cajun Delights'}

In [141]:
new_chain("get the names of the suppliers according to the Country wise")



> Entering new SQLDatabaseChain chain...
get the names of the suppliers according to the Country wise
SQLQuery:SELECT CompanyName, Country FROM Supplier GROUP BY Country;

ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Column 'Supplier.CompanyName' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause. (8120) (SQLExecDirectW)")
[SQL: SELECT CompanyName, Country FROM Supplier GROUP BY Country;]
(Background on this error at: https://sqlalche.me/e/20/f405)